In [1]:
import re
import string
import data_utils

In [65]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client['real_estate_data']
read_collection = db['real_estate_query_data']
write_collection = db['real_estate_query_data_normailze']

In [2]:
def normalize(text, text_type="none"):
    text = data_utils.compound2unicode(text)
    text = re.sub('Mười','10', text)
    text = re.sub('mười','10', text)
    text = data_utils.remove_vietnamese_accent(text)
    print('haha', text)
    if text_type=="price":
        low, high = normalize_price(text)
        return low, high
    
    elif text_type=="area":
        text = normalize_area(text)
        
    elif text_type=="ward":
        text = normalize_ward(text)
        
    return text

In [13]:
main_divider = '-'
dividers = ['toi', 'va', '~', 'hoac']
currency_unit = ['ti', 'ty', 'trieu', 'tr', 'nghin', 'ngan', 'k']
maping_num = {'mot':'1', 'hai':'2', 'ba':'3', 'bon':'4', 'nam':'5', 'sau':'6', 'bay':'7', 'tam':'8', 'chin':'9'}
def normalize_price(text):
    for key, value in maping_num.items():
         text = re.sub(key, value, text)
    text = re.sub('ti', 'ty', text)    
    text = re.sub('tram', '#00', text)
    text = re.sub('trieu', 'tr', text)
    text = re.sub('muoi', '#0', text) #Mươi :)))
    text = re.sub(' #', '', text)
    text = re.sub('mot', '1', text) #Mốt nhé
    text = re.sub('k', 'kk', text)
    text = re.sub('nghin', 'kk', text)
    text = re.sub('ngan', 'kk', text)
    text = re.sub(',', '.', text)
    
    for div in dividers:
        text = re.sub(div, main_divider, text)
    price_list = list()
    arr = text.split(main_divider)
    print('Divided: ', arr)
    biggest_unit = None
    for element in reversed(arr):
        prices = powerful_split(element)
        print('Splittt: ', prices)
        for price in reversed(prices):
            value, unit = normalize_price_unit(price, biggest_unit)
            price_list.append(value)
            biggest_unit = unit
    
    low, high = min(price_list), max(price_list)
    if low == high:
        high = None
    return low, high
    
#     pass
#     1  2 tỉ
    # 1 tỉ 200 tr 800 nghi 2 tỉ 300tr
#     1 2 ti 200 tr
#     1 ti 1 ti 300 tr

re_num = '\d+(\s.\s\d+)?'
re_hud = re.compile('(\d+(\s.\s\d+)?\skk)')
re_mil = re.compile('(\d+(\s.\s\d+)?\str)')
re_bil = re.compile('(\d+(\s.\s\d+)?\sty)')


def powerful_split(text):
    text = text.strip()
    idx_bil = [0] + [i.start() for i in re.finditer(re_bil, text)] + [len(text)]
    idx_mil = [0] + [i.start() for i in re.finditer(re_mil, text)] + [len(text)]
    idx_hud = [0] + [i.start() for i in re.finditer(re_hud, text)] + [len(text)]
    print('idx_bil', idx_bil)
    print('idx_mil', idx_mil)
    print('idx_hud', idx_hud)
    price_list = list()
    if len(idx_bil) > 2:
        for i in range(0, len(idx_bil) - 1):
            price = text[idx_bil[i]:idx_bil[i+1]]
            if price != '':
                price_list.append(price)
    elif len(idx_mil) > 2:
        for i in range(0, len(idx_mil) - 1):
            price = text[idx_mil[i]:idx_mil[i+1]]
            if price != '':
                price_list.append(price)
    elif len(idx_hud) > 2:
        for i in range(0, len(idx_hud) - 1):
            price = text[idx_hud[i]:idx_hud[i+1]]
            if price != '':
                price_list.append(price)
    elif text != '':
        price_list.append(text)
        
    return price_list
        
    
maping_unit = {'ty': 1000000000, 'tr': 1000000, 'kk':1000}
def normalize_price_unit(text, pre_unit='kk'):
    if text == '':
        return 0, None
    final_value = 0
    arr = text.split(' ')
    current_unit = pre_unit
    num_list = [float(re.sub(' ','', i.group())) for i in re.finditer('\d+(\s.\s\d+)?', text)]
    unit_list = [i.group() for i in re.finditer('[a-z]+', text)]
    print(num_list)
    print(unit_list)
    if len(unit_list) == 0:
        final_value = num_list[-1] * maping_unit[pre_unit]
        return final_value, pre_unit
    
    odd_unit = 'kk'
    for i in range(min(len(num_list), len(unit_list))):
        num = num_list[i]
        unit = unit_list[i]
        if unit in maping_unit.keys():
            final_value += maping_unit[unit]*num
            odd_unit = unit
    if len(num_list) > len(unit_list):
        odd = num_list[len(unit_list)]
        if odd < 10:
            final_value += maping_unit[odd_unit]*odd/10
        else:
            final_value += maping_unit[odd_unit]*odd/1000
                
    print(final_value, odd_unit)
    return final_value, odd_unit
    

In [15]:
normalize('1 . 2 2 ty', 'price')

haha 1.2 2 ty
Divided:  ['1.2 2 ty']
idx_bil [0, 4, 8]
idx_mil [0, 8]
idx_hud [0, 8]
Splittt:  ['1.2 ', '2 ty']
[2.0]
['ty']
2000000000.0 ty
[1.0, 2.0]
[]


(2000000000.0, None)

In [5]:
,data_utils.remove_vietnamese_accent('2 tỷ')

'2 ty'

In [17]:
print('ahihi\b')

ahih


In [6]:
count = 0
for document in read_collection.find():
#     document["price"] = normalize_price(document["price"])
    print(document["price"])
#     write_collection.insert_one(document)
    count = count+1
    if count > 10000:
        break

: 3 tỷ 950 triệu
: 2 tỷ 300 triệu
: 3 tỷ 550 triệu
: 2 tỷ 900 triệu
: 3 tỷ 800 triệu
: 30 triệu
: 3 tỷ 500 triệu
: 5 tỷ 450 triệu
: 1 tỷ 250 triệu
: 800 triệu
: 4 tỷ 788 triệu
: 1 tỷ 845 triệu
: 2 tỷ 600 triệu
: 330 triệu
: 4 tỷ 700 triệu
: 3 tỷ 80 triệu
: 1 tỷ 750 triệu
: 5 tỷ 300 triệu
: 3 tỷ 450 triệu
: 900 triệu
: 900 triệu
: 3 tỷ 500 triệu
: 5 tỷ 500 triệu
: 2 tỷ 800 triệu
: 1 tỷ 750 triệu
: 2 tỷ 488 triệu
: 3 tỷ 50 triệu
: 1 tỷ 360 triệu
: 1 tỷ 700 triệu
: 3 tỷ 100 triệu
: 1 tỷ 550 triệu
: 3 tỷ 150 triệu
: 2 tỷ 300 triệu
: 1 tỷ 350 triệu
: 1 tỷ 350 triệu
: 1 tỷ 350 triệu
: 1 tỷ 270 triệu
: 988 triệu
: 900 triệu
: 1 tỷ 100 triệu
: 1 tỷ 200 triệu
: 900 triệu
: 1 tỷ 400 triệu
: 1 tỷ 455 triệu
: 1 tỷ 389 triệu
: 1 tỷ 200 triệu
: 900 triệu
: 3 tỷ 900 triệu
: 1 tỷ 900 triệu
: 860 triệu
: 1 tỷ 400 triệu
: 1 tỷ
: 200 triệu
: 3 tỷ 900 triệu
: 2 tỷ 250 triệu
: 3 tỷ 900 triệu
: 2 tỷ 720 triệu
: 3 tỷ 900 triệu
: 3 tỷ 900 triệu
: 1 tỷ 260 triệu
: 4 tỷ 700 triệu
: 1 tỷ 650 triệu
: 2 tỷ 200 tri

: 1 tỷ
: 900 triệu
: 1 tỷ
: 1 tỷ
: 1 tỷ
: 1 tỷ
: 890 triệu
: 1 tỷ 500 triệu
: 1 tỷ 600 triệu
: 1 tỷ
: 1 tỷ
: 1 tỷ
: 1 tỷ 200 triệu
: 1 tỷ
: 1 tỷ
: 10 tỷ 650 triệu
: 1 tỷ
: 1 tỷ
: 1 tỷ
: 1 tỷ 650 triệu
: 3 tỷ
: 3 tỷ 300 triệu
: 2 tỷ
: 3 tỷ 200 triệu
: 1 tỷ 580 triệu
: 1 tỷ 680 triệu
: 1 tỷ 330 triệu
: 1 tỷ 870 triệu
: 1 tỷ 590 triệu
: 1 tỷ 600 triệu
: 1 tỷ 200 triệu
: 800 triệu
: 1 tỷ 550 triệu
: 2 tỷ 500 triệu
: 1 tỷ 600 triệu
: 2 tỷ 300 triệu
: 3 tỷ 400 triệu
: 1 tỷ 600 triệu
: 1 tỷ 550 triệu
: 1 tỷ 450 triệu
: 2 tỷ 50 triệu
: 1 tỷ 350 triệu
: 1 tỷ 694 triệu
: 2 tỷ 650 triệu
: 1 tỷ 780 triệu
: 1 tỷ 920 triệu
: 1 tỷ 350 triệu
: 1 tỷ 400 triệu
: 2 tỷ 50 triệu
: 2 tỷ
: 1 tỷ 350 triệu
: 6 tỷ
: 1 tỷ 350 triệu
: 1 tỷ 800 triệu
: 1 tỷ 700 triệu
: 1 tỷ 850 triệu
: 1 tỷ 700 triệu
: 1 tỷ 600 triệu
: 1 tỷ 500 triệu
: 1 tỷ 730 triệu
: 1 tỷ 950 triệu
: 1 tỷ 950 triệu
: 1 tỷ 720 triệu
: 1 tỷ 750 triệu
: 2 tỷ
: 2 tỷ 800 triệu
: 1 tỷ 850 triệu
: 2 tỷ 450 triệu
: 1 tỷ 350 triệu
: 2 tỷ 450 triệu
: 1 tỷ

: 1 tỷ 600 triệu
: 1 tỷ 600 triệu
: 2 tỷ 500 triệu
: 1 tỷ 450 triệu
: 2 tỷ
: 4 tỷ
: 5 tỷ 500 triệu
: 4 tỷ 500 triệu
: 2 tỷ 800 triệu
: 2 tỷ 500 triệu
: 4 tỷ
: 2 tỷ
: 3 tỷ
: 5 tỷ
: 4 tỷ 700 triệu
: 2 tỷ 850 triệu
: 3 tỷ 150 triệu
: 6 tỷ 140 triệu
: 2 tỷ 50 triệu
: 2 tỷ
: 3 tỷ 300 triệu
: 1 tỷ 700 triệu
: 5 tỷ
: 1 tỷ 580 triệu
: 2 tỷ 50 triệu
: 1 tỷ 350 triệu
: 2 tỷ 450 triệu
: 1 tỷ 550 triệu
: 1 tỷ 650 triệu
: 1 tỷ 450 triệu
: 1 tỷ 720 triệu
: 3 tỷ 700 triệu
: 2 tỷ 400 triệu
: 1 tỷ 450 triệu
: 4 tỷ 200 triệu
: 6 tỷ 400 triệu
: 3 tỷ 200 triệu
: 3 tỷ 400 triệu
: 5 tỷ 100 triệu
: 2 tỷ 650 triệu
: 3 tỷ 900 triệu
: 2 tỷ
: 4 tỷ 500 triệu
: 4 tỷ 750 triệu
: 3 tỷ 200 triệu
: 2 tỷ 500 triệu
: 6 tỷ 200 triệu
: 3 tỷ 500 triệu
: 10 tỷ 500 triệu
: 5 tỷ
: 2 tỷ 450 triệu
: 4 tỷ 800 triệu
: 1 tỷ 800 triệu
: 1 tỷ 500 triệu
: 5 tỷ 200 triệu
: 1 tỷ 470 triệu
: 1 tỷ 600 triệu
: 2 tỷ 450 triệu
: 1 tỷ 585 triệu
: 1 tỷ 650 triệu
: 4 tỷ 220 triệu
: 3 tỷ 500 triệu
: 4 tỷ 100 triệu
: 10 tỷ 500 triệu
: 1 tỷ 870 t

: 1 tỷ 400 triệu
: 2 tỷ 516 triệu
: 1 tỷ 500 triệu
: 1 tỷ 300 triệu
: 4 tỷ 200 triệu
: 4 tỷ 600 triệu
: 3 tỷ 800 triệu
: 4 tỷ 100 triệu
: 3 tỷ 500 triệu
: 3 tỷ 100 triệu
: 4 tỷ 250 triệu
: 2 tỷ 450 triệu
: 1 tỷ 500 triệu
: 2 tỷ 90 triệu
: 4 tỷ
: 4 tỷ 300 triệu
: 5 tỷ 300 triệu
: 5 tỷ 100 triệu
: 2 tỷ 170 triệu
: 3 tỷ 850 triệu
: 855 triệu
: 2 tỷ
: 3 tỷ 250 triệu
: 2 tỷ 650 triệu
: 2 tỷ 950 triệu
: 3 tỷ 300 triệu
: 3 tỷ 850 triệu
: 1 tỷ 560 triệu
: 3 tỷ 350 triệu
: 2 tỷ 400 triệu
: 1 tỷ 300 triệu
: 3 tỷ 500 triệu
: 1 tỷ 500 triệu
: 2 tỷ 450 triệu
: 1 tỷ 800 triệu
: 3 tỷ 200 triệu
: 2 tỷ 250 triệu
: 1 tỷ 720 triệu
: 1 tỷ 700 triệu
: 1 tỷ 730 triệu
: 2 tỷ 530 triệu
: 10 tỷ 500 triệu
: 1 tỷ 350 triệu
: 5 tỷ 500 triệu
: 1 tỷ 550 triệu
: 5 tỷ 100 triệu
: 2 tỷ 320 triệu
: 3 tỷ 550 triệu
: 2 tỷ 950 triệu
: 1 tỷ 670 triệu
: 1 tỷ 200 triệu
: 1 tỷ 850 triệu
: 1 tỷ 950 triệu
: 1 tỷ 350 triệu
: 1 tỷ 550 triệu
: 3 tỷ 100 triệu
: 1 tỷ 500 triệu
: 1 tỷ 800 triệu
: 1 tỷ 950 triệu
: 1 tỷ 650 triệu
: 1 t

: 2 tỷ 350 triệu
: 1 tỷ 320 triệu
: 5 tỷ 100 triệu
: 1 tỷ 950 triệu
: 3 tỷ
: 3 tỷ 740 triệu
: 3 tỷ 500 triệu
: 504 triệu
: 1 tỷ 600 triệu
: 1 tỷ 280 triệu
: 940 triệu
: 1 tỷ 800 triệu
: 1 tỷ 950 triệu
: 8 tỷ 500 triệu
: 3 tỷ 150 triệu
: 2 tỷ
: 2 tỷ
: 2 tỷ 880 triệu
: 6 tỷ 600 triệu
: 1 tỷ 900 triệu
: 2 tỷ 150 triệu
: 3 tỷ 838 triệu
: 1 tỷ 300 triệu
: 1 tỷ 270 triệu
: 1 tỷ 426 triệu
: 1 tỷ 650 triệu
: 1 tỷ 370 triệu
: 1 tỷ 200 triệu
: 1 tỷ 400 triệu
: 1 tỷ 610 triệu
: 3 tỷ 199 triệu 999 nghìn
: 4 tỷ
: 300 triệu
: 3 tỷ
: 1 tỷ 700 triệu
: 15 tỷ
: 2 tỷ 999 triệu
: 350 triệu
: 3 tỷ 100 triệu
: 1 tỷ 680 triệu
: 1 tỷ 78 triệu
: 3 tỷ 200 triệu
: 5 tỷ 300 triệu
: 2 tỷ
: 2 tỷ 500 triệu
: 4 tỷ 300 triệu
: 2 tỷ 500 triệu
: 1 tỷ 850 triệu
: 2 tỷ 600 triệu
: 6 tỷ 800 triệu
: 1 tỷ 400 triệu
: 1 tỷ 850 triệu
: 12 tỷ
: 5 tỷ 616 triệu
: 2 tỷ 300 triệu
: 764 triệu
: 792 triệu
: 5 tỷ 246 triệu 397 nghìn
: 25 tỷ 300 triệu
: 700 triệu
: 1 tỷ 750 triệu
: 3 tỷ 100 triệu
: 4 tỷ 50 triệu
: 1 tỷ 568 triệu
: 1 tỷ

: 750 triệu
: 2 tỷ 400 triệu
: 6 tỷ 800 triệu
: 370 triệu
: 1 tỷ 500 triệu
: 650 triệu
: 8 tỷ 300 triệu
: 1 tỷ 200 triệu
: 1 tỷ 900 triệu
: 2 tỷ
: 2 tỷ 290 triệu
: 925 triệu
: 3 tỷ 750 triệu
: 3 tỷ 100 triệu
: 11 tỷ
: 2 tỷ 350 triệu
: 3 tỷ 350 triệu
: 2 tỷ 400 triệu
: 850 triệu
: 560 triệu
: 560 triệu
: 1 tỷ 500 triệu
: 736 triệu
: 450 triệu
: 1 tỷ 125 triệu
: 61 tỷ
: 3 tỷ 250 triệu
: 3 tỷ 250 triệu
: 930 triệu
: 468 triệu
: 58 tỷ
: 3 tỷ 250 triệu
: 735 triệu
: 499 triệu
: 860 triệu
: 890 triệu
: 890 triệu
: 750 triệu
: 870 triệu
: 1 tỷ 350 triệu
: 1 tỷ 100 triệu
: 860 triệu
: 639 triệu
: 650 triệu
: 1 tỷ 500 triệu
: 790 triệu
: 1 tỷ 350 triệu
: 989 triệu
: 860 triệu
: 650 triệu
: 980 triệu
: 839 triệu
: 780 triệu
: 1 tỷ 80 triệu
: 1 tỷ 300 triệu
: 1 tỷ 439 triệu
: 797 triệu
: 870 triệu
: 1 tỷ 400 triệu
: 639 triệu
: 1 tỷ 280 triệu
: 1 tỷ 700 triệu
: 980 triệu
: 900 triệu
: 790 triệu
: 3 tỷ 800 triệu
: 700 triệu
: 989 triệu
: 697 triệu
: 850 triệu
: 850 triệu
: 790 triệu
: 852 triệu
: 

: 799 triệu
: 350 triệu
: 1 tỷ 110 triệu
: 1 tỷ 209 triệu
: 3 tỷ 800 triệu
: 2 tỷ 30 triệu
: 700 triệu
: 32 tỷ
: 579 triệu
: 11 tỷ 475 triệu
: 1 tỷ
: 3 tỷ 800 triệu
: 380 triệu
: 4 tỷ
: 469 triệu
: 523 triệu
: 650 triệu
: 2 tỷ 150 triệu
: 6 tỷ 900 triệu
: 480 triệu
: 449 triệu
: 350 triệu
: 600 triệu
: 4 tỷ
: 650 triệu
: 599 triệu
: 800 triệu
: 980 triệu
: 540 triệu
: 310 triệu
: 600 triệu
: 345 triệu
: 750 triệu
: 7 tỷ
: 430 triệu
: 379 triệu
: 550 triệu
: 400 triệu
: 1 tỷ 850 triệu
: 650 triệu
: 3 tỷ 800 triệu
: 950 triệu
: 579 triệu
: 700 triệu
: 550 triệu
: 1 tỷ 200 triệu
: 900 triệu
: 450 triệu
: 450 triệu
: 479 triệu
: 800 triệu
: 600 triệu
: 720 triệu
: 579 triệu
: 650 triệu
: 599 triệu
: 780 triệu
: 750 triệu
: 6 tỷ
: 319 triệu
: 4 tỷ 950 triệu
: 279 triệu
: 3 tỷ 600 triệu
: 660 triệu
: 980 triệu
: 7 tỷ 400 triệu
: 299 triệu
: 563 triệu
: 750 triệu
: 3 tỷ 860 triệu
: 600 triệu
: 2 tỷ 995 triệu 200 nghìn
: 11 tỷ 900 triệu
: 600 triệu
: 400 triệu
: 720 triệu
: 400 triệu
: 279 tri

: 450 triệu
: 2 tỷ 100 triệu
: 709 triệu
: 590 triệu
: 600 triệu
: 350 triệu
: 1 tỷ 200 triệu
: 390 triệu
: 350 triệu
: 3 tỷ 750 triệu
: 2 tỷ 350 triệu
: 950 triệu
: 1 tỷ 200 triệu
: 679 triệu
: 820 triệu
: 230 triệu
: 2 tỷ 800 triệu
: 109 triệu
: 1 tỷ 200 triệu
: 760 triệu
: 950 triệu
: 950 triệu
: 400 triệu
: 420 triệu
: 370 triệu
: 3 tỷ
: 990 triệu
: 850 triệu
: 450 triệu
: 800 triệu
: 600 triệu
: 559 triệu
: 550 triệu
: 750 triệu
: 700 triệu
: 650 triệu
: 1 tỷ 109 triệu
: 850 triệu
: 2 tỷ 800 triệu
: 630 triệu
: 400 triệu
: 1 tỷ 100 triệu
: 479 triệu
: 600 triệu
: 11 tỷ
: 510 triệu
: 750 triệu
: 580 triệu
: 900 triệu
: 6 tỷ
: 950 triệu
: 870 triệu
: 1 tỷ 600 triệu
: 850 triệu
: 800 triệu
: 870 triệu
: 380 triệu
: 382 triệu
: 390 triệu
: 3 tỷ 300 triệu
: 690 triệu
: 400 triệu
: 650 triệu
: 1 tỷ 500 triệu
: 1 tỷ 200 triệu
: 390 triệu
: 10 tỷ 500 triệu
: 850 triệu
: 700 triệu
: 590 triệu
: 480 triệu
: 16 tỷ 500 triệu
: 450 triệu
: 420 triệu
: 850 triệu
: 900 triệu
: 330 triệu
: 13 tỷ


: 1 tỷ 674 triệu
: 3 tỷ 800 triệu
: 930 triệu
: 650 triệu
: 16 tỷ 200 triệu
: 14 tỷ 625 triệu
: 860 triệu
: 860 triệu
: 650 triệu
: 6 tỷ 600 triệu
: 650 triệu
: 1 tỷ 680 triệu
: 1 tỷ 680 triệu
: 238 tỷ 680 triệu
: 850 triệu
: 1 tỷ 346 triệu 200 nghìn
: 1 tỷ 330 triệu 300 nghìn
: 2 tỷ 250 triệu
: 3 tỷ
: 2 tỷ 550 triệu
: 650 triệu
: 689 triệu
: 2 tỷ 500 triệu
: 2 tỷ 500 triệu
: 3 tỷ 300 triệu
: 172 triệu
: 2 tỷ 70 triệu
: 6 tỷ 700 triệu
: 4 tỷ 650 triệu
: 2 tỷ 850 triệu
: 3 tỷ 500 triệu
: 2 tỷ
: 3 tỷ
: 2 tỷ 600 triệu
: 1 tỷ 250 triệu
: 2 tỷ 850 triệu
: 3 tỷ 300 triệu
: 2 tỷ 400 triệu
: 3 tỷ 400 triệu
: 1 tỷ 150 triệu
: 689 triệu
: 800 triệu
: 1 tỷ 500 triệu
: 1 tỷ 500 triệu
: 2 tỷ 400 triệu
: 880 triệu
: 2 tỷ 500 triệu
: 1 tỷ 800 triệu
: 1 tỷ 750 triệu
: 2 tỷ 400 triệu
: 2 tỷ 80 triệu
: 3 tỷ 300 triệu
: 5 tỷ 500 triệu
: 2 tỷ 400 triệu
: 1 tỷ 500 triệu
: 2 tỷ 250 triệu
: 1 tỷ 500 triệu
: 2 tỷ 400 triệu
: 3 tỷ 300 triệu
: 3 tỷ 250 triệu
: 2 tỷ 250 triệu
: 1 tỷ 950 triệu
: 2 tỷ 550 triệu
: 

In [ ]:
2 tỷ 200 triệu 800 nghìn 
2 tỷ 200 triệu 800 nghìn  tới 2 tỷ 200 triệu 800 nghìn 
